In [1]:
# Import Data Manipulation Libraries
import numpy as np
import pandas as pd

# Import Data Visualization Libraries
import matplotlib.pyplot as plt
import seaborn as sns

# Import Filter Warning Libraries
import warnings
warnings.filterwarnings('ignore')

# Import Logging  Files
import logging

logging.basicConfig(
    level=logging.INFO,
    filemode='w',
    filename='model.log',
    format='%(asctime)s - %(levelname)s - %(message)s')


# import ML Libraries
from sklearn.preprocessing import StandardScaler,MinMaxScaler, RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report

In [2]:
logging.info('importing data set using pandas function')

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
df.corr().style.background_gradient(cmap='coolwarm')

In [ ]:
df.Crop.unique()

In [ ]:
from collections import OrderedDict

stats=[]

for i in df.columns:
    if df[i].dtype!='object':
     numerical_stats= OrderedDict({
        'feature':i,
        'unique_values':df[i].nunique(),
        'mode':df[i].mode()[0],
        'mean':df[i].mean(),
        'std_deviation':df[i].std(),
        'min_value':df[i].min(),
        'max_value':df[i].max(),
        'range':df[i].max() - df[i].min(),
        'skewness':df[i].skew(),
        'kurtosis':df[i].kurtosis(),
        'IQR':df[i].quantile(0.75) - df[i].quantile(0.25),
    })
    stats.append(numerical_stats)

df_stats=pd.DataFrame(stats)

df_stats



In [ ]:
from sklearn.preprocessing import LabelEncoder

df['Crop']=LabelEncoder().fit_transform(df['Crop'])


In [ ]:
X=df.drop(columns='Crop')
y=df['Crop']

y

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=42)


In [ ]:
plt.figure(figsize= (20,10))
sns.boxenplot(data = df)
plt.xticks(rotation = 90)
plt.show()

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler=MinMaxScaler()
X_train=scaler.fit_transform(X_train)

X_test=scaler.transform(X_test)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
RF=RandomForestClassifier()
RF.fit(X_train,y_train)

y_pred_RF=RF.predict(X_test)

from sklearn.metrics import accuracy_score

accuracy_score(y_test,y_pred_RF)



In [ ]:
from sklearn.ensemble import RandomForestRegressor
RF=RandomForestClassifier()
RF.fit(X_train,y_train)

y_pred_RFR=RF.predict(X_test)

from sklearn.metrics import r2_score

r2_score(y_test,y_pred_RFR)

In [ ]:
from sklearn.svm import SVC
svc=SVC()

svc.fit(X_train,y_train)

y_pred_SVC=svc.predict(X_test)

from sklearn.metrics import accuracy_score

accuracy_score(y_test,y_pred_SVC)   
